*This code downloads CSVs and GeoTiffs that can be used for creating predictions DFs and mapping original variables (should we create predictions dfs here?)*

In [2]:
# set up GEE
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

# define area
region = ee.FeatureCollection("USDOS/LSIB/2017").filter(ee.Filter.eq("COUNTRY_NA", "Brazil"));

# brazil pixels for DF
brazilPixels = ee.FeatureCollection('users/cglidden/1km_grid_brazil');

# Create a source image where the geometry is 1, everything else is 0 - for cost distance mapping, unmasking, etc.
sources = ee.Image().toByte().paint(region.geometry(), 1);

# mask the sources image with itself.
sources = sources.selfMask();

# create land mask (remove ocean from data by masking)
mask = ee.Image.constant(1).clip(region).mask();



To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=qZdP24rXax5WHTS-r2a1Ac4YoFY1RC534WJgLX2umic&tc=jrRB6vDZaOCievqobw5jno8zwgdaFgJrb7zltdU02Hw&cc=5a10B9qB-qkT5TUYup1G5dWpisumFBDQ0MOEE5I3BvM

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXmtYS_FiU_VDob6JkyiNr3QudiADKLGo4Zt3CjkPwfbYKEj3wHWJfU

Successfully saved authorization token.


*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_doiqkQG3NJ1t8IS?source=API


**BASE**

In [ ]:
# function for riverDist cost mapping

sheds = ee.FeatureCollection("WWF/HydroSHEDS/v1/FreeFlowingRivers").filterMetadata('COUNTRY', 'equals', 'Brazil');

orderSheds = sheds.filterMetadata('RIV_ORD', 'not_greater_than', 8);

ordShed_im = (orderSheds.reduceToImage(
                    properties = ['BB_ID'],
                    reducer = ee.Reducer.first()
                    ).multiply(0).rename('BB_ID').unmask(sources));

overRivers = ee.Image().toByte().paint(region.geometry(), 20000);



In [ ]:
elevation = (ee.Image('NASA/NASADEM_HGT/001').select('elevation')
            .resample("bilinear")
            .reproject(crs='EPSG:4326', scale=1000)
            .reduce(ee.Reducer.mean())
            .clip(region).
            rename('elevation'));

bulk = (ee.Image("OpenLandMap/SOL/SOL_BULKDENS-FINEEARTH_USDA-4A1H_M/v02").select('b0').rename('bulk')
            .reproject(crs='EPSG:4326', scale=1000)
            .reduce(ee.Reducer.mean())
            .clip(region)
            .rename('bulk'));

clay = (ee.Image("OpenLandMap/SOL/SOL_CLAY-WFRACTION_USDA-3A1A1A_M/v02").select('b0')
            .reproject(crs='EPSG:4326', scale=1000)
            .reduce(ee.Reducer.mean())
            .clip(region)
            .rename('clay'));

pH = (ee.Image("OpenLandMap/SOL/SOL_PH-H2O_USDA-4C1A2A_M/v02").select('b0').rename('pH')
            .reproject(crs='EPSG:4326', scale=1000)
            .reduce(ee.Reducer.mean())
            .clip(region)
            .rename('pH'));

sand = (ee.Image("OpenLandMap/SOL/SOL_SAND-WFRACTION_USDA-3A1A1A_M/v02").select('b0').rename('sand')
            .reproject(crs='EPSG:4326', scale=1000)
            .reduce(ee.Reducer.mean())
            .clip(region)
            .rename('sand'));

soilWater = (ee.Image("OpenLandMap/SOL/SOL_WATERCONTENT-33KPA_USDA-4B1C_M/v01").select('b0').rename('soilWater')
            .reproject(crs='EPSG:4326', scale=1000)
            .reduce(ee.Reducer.mean())
            .clip(region)
            .rename('soilWater'));

soilCarbon = (ee.Image("OpenLandMap/SOL/SOL_ORGANIC-CARBON_USDA-6A1C_M/v02").select('b0').rename('soilCarbon')
            .reproject(crs='EPSG:4326', scale=1000)
            .reduce(ee.Reducer.mean())
            .clip(region)
            .rename('soilCarbon'));

MERIT = ee.Image('MERIT/Hydro/v1_0_1');

upa = (MERIT.select('upa')
      .double()
      .reproject(crs='EPSG:4326', scale=1000)
      .reduce(ee.Reducer.mean())
      .clip(region)
      .rename('upa'));

hnd = (MERIT.select('hnd')
      .double()
      .reproject(crs='EPSG:4326', scale=1000)
      .reduce(ee.Reducer.max())
      .clip(region)
      .rename('hnd'));

aspect = (ee.Terrain.aspect(elevation)
          .double()
          .reproject(crs='EPSG:4326', scale=1000)
          .reduce(ee.Reducer.mean())
          .clip(region)
          .rename('aspect'));

riverDist = (ordShed_im.cumulativeCost(
  source = ordShed_im.eq(0),
  maxDistance = 10 * 1000
  ).rename('riverDist').double().unmask(overRivers));

waterJRC = ee.Image('JRC/GSW1_3/GlobalSurfaceWater');

waterOccur = (waterJRC.select('occurrence').unmask(sources.multiply(0)).updateMask(mask)
            .reproject(crs='EPSG:4326', scale=1000)
            .reduce(ee.Reducer.mean())
            .clip(region)
            .rename('waterOccur'));

waterReoccur = (waterJRC.select('recurrence').unmask(sources.multiply(0)).updateMask(mask)
                .reproject(crs='EPSG:4326', scale=1000)
                .reduce(ee.Reducer.mean())
                .clip(region)
                .rename('waterRecur'));

**Stack and export**

In [ ]:
base_vars = (elevation.addBands([bulk, clay, pH, sand, soilWater, soilCarbon,
                                upa, hnd, aspect, riverDist, waterOccur,waterReoccur ]));


base_vars_task = ee.batch.Export.image.toDrive(
    image=base_vars,
    description='schisto_base_variables',
    folder='final_schisto_data',
    region=region.geometry(),
    scale=1000,
    crs='EPSG:4326'
)
base_vars_task.start()

# Reduce the regions and export the table
# base_vars_table = base_vars.reduceRegions(
#    collection=brazilPixels,
#    reducer=ee.Reducer.mean(),
#    scale=1000,
#    tileScale=16
# )

# Define the export parameters
# task = ee.batch.Export.table.toDrive(
#    collection = base_vars_table,
#    description = 'prediction_base_vars',
#    folder = 'final_schisto_data',
#    fileFormat ='CSV'
# )

# Start the export task
# task.start()



NameError: ignored

**CLIMATE** (change to double)

* 1992

In [12]:
chelsa_1992 = (ee.ImageCollection("users/cglidden/CHELSA_decadal_mean/1991_2000/CHELSA_1991_2000")
              .toBands()
              .double()
              .reproject(crs='EPSG:4326', scale=1000)
              .select('bio2_1991_b1', 'bio3_1991_b1', 'bio8_1991_b1', 'bio9_1991_b1', 'bio12_1991_b1', 'bio13_1991_b1', 'bio14_1991_b1', 'bio15_1991_b1', 'bio16_1991_b1')
              .rename('bio2', 'bio3', 'bio8', 'bio9', 'bio12', 'bio13', 'bio14', 'bio15', 'bio16')
              .clip(region))


* 2017

In [13]:
chelsa_2017 = (ee.ImageCollection("users/cglidden/CHELSA_decadal_mean/2011_2017/CHELSA_2011_2017")
              .toBands()
              .double()
              .reproject(crs='EPSG:4326', scale=1000)
              .select('bio2_2011_b1', 'bio3_2011_b1', 'bio8_2011_b1', 'bio9_2011_b1', 'bio12_2011_b1', 'bio13_2011_b1', 'bio14_2011_b1', 'bio15_2011_b1', 'bio16_2011_b1')
              .rename('bio2', 'bio3', 'bio8', 'bio9', 'bio12', 'bio13', 'bio14', 'bio15', 'bio16')
              .clip(region))

**URBAN**

* 1992

In [14]:
# dist urban & high urban AR

urban_dist_1992 = (ee.Image('projects/gbsc-gcp-lab-emordeca/assets/urban_mapping/urbanCostMapARinterpolated').select('inte_1992')
            .reproject(crs='EPSG:4326', scale=1000)
            .reduce(ee.Reducer.mean())
            .clip(region)
            .rename('ar_urban_gradient')); # note that this is actually just distance

high_urban_dist_1992 = (ee.Image('projects/gbsc-gcp-lab-emordeca/assets/urban_mapping/urbanCostMapARinterpolated').select('high_1992')
            .reproject(crs='EPSG:4326', scale=1000)
            .reduce(ee.Reducer.mean())
            .clip(region)
            .rename('high_urban_gradient')); # note that this is actually just distance

* 2017

In [15]:
# dist urban & high urban AR
urban_dist_2017 = (ee.Image('projects/gbsc-gcp-lab-emordeca/assets/urban_mapping/urbanCostMapAR').select('inte_2017')
            .reproject(crs='EPSG:4326', scale=1000)
            .reduce(ee.Reducer.mean())
            .clip(region)
            .rename('ar_urban_gradient')); # note that this is actually just distance

high_urban_dist_2017 = (ee.Image('projects/gbsc-gcp-lab-emordeca/assets/urban_mapping/urbanCostMapAR').select('high_2017')
            .reproject(crs='EPSG:4326', scale=1000)
            .reduce(ee.Reducer.mean())
            .clip(region)
            .rename('high_urban_gradient')); # note that this is actually just distance


**RURAL**

In [17]:
rural_image1992 = ee.Image('projects/gbsc-gcp-lab-emordeca/assets/urban_mapping/ruralCostMapInterpolated').select(['rural_1992'])
rural_image2017 = ee.Image('projects/gbsc-gcp-lab-emordeca/assets/urban_mapping/ruralCostMap').select(['rural_2017'])
# rural_image = rural_image1992.addBands(rural_image2017)

# rural_task = ee.batch.Export.image.toDrive(
#    image=rural_image,
#    description='rural_distance',
#    folder='final_schisto_data',
#    region=region.geometry(),
#    scale=1000,
#    crs='EPSG:4326'
#)
# rural_task.start()

**LULC** (this was obtained by running MAPBIOMAS for every grid cell of Brazil and then rasterizing the DF)



*   1992




In [18]:
temp_crops_1992 =(ee.Image('projects/gbsc-gcp-lab-emordeca/assets/land_cover/temp_crops_1992').select('b1')
            .double()
            .reproject(crs='EPSG:4326', scale=1000)
            .reduce(ee.Reducer.mean())
            .clip(region)
            .rename('temp_crops'));

ag_mosiac_1992 =(ee.Image('projects/gbsc-gcp-lab-emordeca/assets/land_cover/ag_mosiac_1992').select('b1')
            .double()
            .reproject(crs='EPSG:4326', scale=1000)
            .reduce(ee.Reducer.mean())
            .clip(region)
            .rename('ag_mosiac'));



*   2017




In [19]:
temp_crops_2017 =(ee.Image('projects/gbsc-gcp-lab-emordeca/assets/land_cover/temp_crops_2017').select('b1')
            .double()
            .reproject(crs='EPSG:4326', scale=1000)
            .reduce(ee.Reducer.mean())
            .clip(region)
            .rename('temp_crops'));

ag_mosiac_2017 =(ee.Image('projects/gbsc-gcp-lab-emordeca/assets/land_cover/ag_mosiac_2017').select('b1')
            .double()
            .reproject(crs='EPSG:4326', scale=1000)
            .reduce(ee.Reducer.mean())
            .clip(region)
            .rename('ag_mosiac'));

**Export Observed Distribution DF (1992)**

In [20]:
observed_1992 = chelsa_1992.addBands([urban_dist_1992, high_urban_dist_1992, rural_image1992, temp_crops_1992, ag_mosiac_1992])

observed_1992_task = ee.batch.Export.image.toDrive(
    image=observed_1992,
    description='observed_1992_climate_urban',
    folder='final_schisto_data',
    region=region.geometry(),
    scale=1000,
    crs='EPSG:4326'
)
observed_1992_task.start()

**Export Observed Distribution DF (2017)**

In [21]:
observed_2017 = chelsa_2017.addBands([urban_dist_2017, high_urban_dist_2017, rural_image2017, temp_crops_2017, ag_mosiac_2017])

observed_2017_task = ee.batch.Export.image.toDrive(
    image=observed_2017,
    description='observed_2017_climate_urban',
    folder='final_schisto_data',
    region=region.geometry(),
    scale=1000,
    crs='EPSG:4326'
)
observed_2017_task.start()

# Reduce the regions and export the table
# observed_2017_table = observed_2017.reduceRegions(
#    collection=brazilPixels,
#    reducer=ee.Reducer.mean(),
#    scale=1000,
#    tileScale=16
# )

# Define the export parameters
# task = ee.batch.Export.table.toDrive(
#    collection = observed_2017_table,
#    description = 'prediction_df_observed_2017',
#    folder = 'final_schisto_data',
#    fileFormat ='CSV'
#)

# Start the export task
# task.start()

**Export Climate Counterfactual DF**

In [22]:
climate_counterfactual = chelsa_1992.addBands([urban_dist_2017, high_urban_dist_2017, rural_image2017, temp_crops_2017, ag_mosiac_2017])

climate_counterfactual_task = ee.batch.Export.image.toDrive(
    image=climate_counterfactual,
    description='climate_counterfactual_vars',
    folder='final_schisto_data',
    region=region.geometry(),
    scale=1000,
    crs='EPSG:4326'
)
climate_counterfactual_task.start()

**Export Urban Counterfactual DF**

In [23]:
urban_counterfactual = chelsa_2017.addBands([urban_dist_1992, high_urban_dist_1992, rural_image1992, temp_crops_2017, ag_mosiac_2017])

urban_counterfactual_task = ee.batch.Export.image.toDrive(
    image=urban_counterfactual,
    description='urban_counterfactual_vars',
    folder='final_schisto_data',
    region=region.geometry(),
    scale=1000,
    crs='EPSG:4326'
)
urban_counterfactual_task.start()